# Part I: Data Extraction

In [1]:
import praw
import pandas as pd
import time

In [2]:
def data_fetch(sub_red_lst = "Jobs", limit = 1000, time_filter = "all"):
    reddit = praw.Reddit(client_id="HB-6S3t-mU0H4JnFryAcAA", 
                         client_secret="iOPDBfwO35IWt4Wd3lbPTbKbZlS0DQ",
                         redirect_uri="http://localhost:8080", 
                         user_agent="Poliscimania")
    
    data = reddit.subreddit(sub_red_lst).top(time_filter = time_filter, limit = limit)
    
    update_data = []
    
    for d in data:
        d_data = {
            "ID": d.id,
            "Subreddit": str(d.subreddit),
            "Title": d.title,
            "Upvote": d.upvote_ratio,
            "Score": d.score,
            "Nums of Comment": d.num_comments
        }
        
        # Fetch post comments for sub-reddit posts
        posts = reddit.submission(d.id)
        
        count = 0
        
        while count < 0:
            try:
                posts.comments.replace_more(limit = None)
                break
            except praw.exceptions.RequestException:
                count += 1
                time.speel(5 * count)
        
        for c in posts.comments.list():
            if isinstance(c, praw.models.MoreComments):
                continue
            d_data["UserID"] = str(c.author)
            d_data["Comment"] = c.body
            update_data.append(d_data.copy())
            
            time.sleep(2)
            
    return pd.DataFrame(update_data)

# Part II: Get Data into DataFrame

In [ ]:
data = data_fetch(sub_red_lst = "cscareerquestions+Jobs", 
                                             limit = 2000, 
                                             time_filter = "all")

# Part III: Export to CSV File

In [ ]:
data.to_csv("DATA.csv", header=True, index=False)